# Bài 1
**Giải Thuật Apriori**

In [10]:
# Hàm để tạo các mục phổ biến kích thước 1 từ tập dữ liệu
def create_candidates(data):
    candidates = set()
    for transaction in data:
        for item in transaction:
            candidates.add(frozenset([item]))
    return candidates

# Hàm để loại bỏ các mục không phổ biến
def prune_candidates(candidates, data, min_support):
    candidate_counts = {}
    for transaction in data:
        for candidate in candidates:
            if candidate.issubset(transaction):
                candidate_counts[candidate] = candidate_counts.get(candidate, 0) + 1

    num_transactions = len(data)
    frequent_candidates = []
    support_data = {}
    for candidate, count in candidate_counts.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_candidates.append(candidate)
        support_data[candidate] = support

    return frequent_candidates, support_data

# Hàm để tạo các tập ứng viên mới từ các tập phổ biến cũ
def create_new_candidates(old_candidates, k):
    new_candidates = []
    num_old_candidates = len(old_candidates)
    for i in range(num_old_candidates):
        for j in range(i + 1, num_old_candidates):
            itemset1 = list(old_candidates[i])[:k - 2]
            itemset2 = list(old_candidates[j])[:k - 2]
            itemset1.sort()
            itemset2.sort()
            if itemset1 == itemset2:
                new_candidates.append(old_candidates[i] | old_candidates[j])
    return new_candidates

# Hàm chính để thực hiện giải thuật Apriori
def apriori(data, min_support):
    candidates = create_candidates(data)
    frequent_items, support_data = prune_candidates(candidates, data, min_support)
    all_frequent_items = [frequent_items]
    k = 2
    while len(all_frequent_items[-1]) > 0:
        new_candidates = create_new_candidates(all_frequent_items[-1], k)
        frequent_items, new_support_data = prune_candidates(new_candidates, data, min_support)
        support_data.update(new_support_data)
        all_frequent_items.append(frequent_items)
        k += 1
    return all_frequent_items, support_data
data = [['milk', 'bread', 'butter'],
        ['milk', 'bread'],
        ['milk', 'butter'],
        ['milk', 'bread', 'butter'],
        ['bread', 'butter']]

min_support = 0.1

frequent_itemsets, support_data = apriori(data, min_support)

print("Các tập mặt hàng phổ biến:")
for k, itemsets in enumerate(frequent_itemsets):
    print(f"Kích thước {k+1}: {itemsets}")

print("\nDữ liệu hỗ trợ:")
for itemset, support in support_data.items():
    print(f"{itemset}: {support}")
# in tập luật kết hợp
def generate_rules(frequent_itemsets, support_data, min_confidence):
    rules = []
    for k_itemset in frequent_itemsets[1:]:
        for itemset in k_itemset:
            for item in itemset:
                antecedent = itemset - set([item])
                consequent = set([item])
                confidence = support_data[itemset] / support_data[antecedent]
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence))
    return rules
min_confidence = 0.5
rules = generate_rules(frequent_itemsets, support_data, min_confidence)
# In ra các luật kết hợp
print("Các luật kết hợp:")
for antecedent, consequent, confidence in rules:
    print(f"{list(antecedent)} => {list(consequent)} (Confidence: {confidence})")



Các tập mặt hàng phổ biến:
Kích thước 1: [frozenset({'butter'}), frozenset({'milk'}), frozenset({'bread'})]
Kích thước 2: [frozenset({'butter', 'milk'}), frozenset({'butter', 'bread'}), frozenset({'bread', 'milk'})]
Kích thước 3: [frozenset({'butter', 'bread', 'milk'})]
Kích thước 4: []

Dữ liệu hỗ trợ:
frozenset({'butter'}): 0.8
frozenset({'milk'}): 0.8
frozenset({'bread'}): 0.8
frozenset({'butter', 'milk'}): 0.6
frozenset({'butter', 'bread'}): 0.6
frozenset({'bread', 'milk'}): 0.6
frozenset({'butter', 'bread', 'milk'}): 0.4
Các luật kết hợp:
['milk'] => ['butter'] (Confidence: 0.7499999999999999)
['butter'] => ['milk'] (Confidence: 0.7499999999999999)
['bread'] => ['butter'] (Confidence: 0.7499999999999999)
['butter'] => ['bread'] (Confidence: 0.7499999999999999)
['milk'] => ['bread'] (Confidence: 0.7499999999999999)
['bread'] => ['milk'] (Confidence: 0.7499999999999999)
['bread', 'milk'] => ['butter'] (Confidence: 0.6666666666666667)
['butter', 'milk'] => ['bread'] (Confidence: 0.66

In [11]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

encoder = TransactionEncoder()
transactions_encoded = encoder.fit(data).transform(data)
df = pd.DataFrame(transactions_encoded, columns=encoder.columns_)
frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)
print(frequent_itemsets)
#in ra các  luật kết hợp
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
for i in rules.index:
    if rules.iloc[i]['confidence'] >= min_confidence:
        print(f"{list(rules.iloc[i]['antecedents'])} => {list(rules.iloc[i]['consequents'])} (Confidence: {rules.iloc[i]['confidence']})")

   support               itemsets
0      0.8                (bread)
1      0.8               (butter)
2      0.8                 (milk)
3      0.6        (butter, bread)
4      0.6          (bread, milk)
5      0.6         (butter, milk)
6      0.4  (butter, bread, milk)
['butter'] => ['bread'] (Confidence: 0.7499999999999999)
['bread'] => ['butter'] (Confidence: 0.7499999999999999)
['bread'] => ['milk'] (Confidence: 0.7499999999999999)
['milk'] => ['bread'] (Confidence: 0.7499999999999999)
['butter'] => ['milk'] (Confidence: 0.7499999999999999)
['milk'] => ['butter'] (Confidence: 0.7499999999999999)
['butter', 'bread'] => ['milk'] (Confidence: 0.6666666666666667)
['butter', 'milk'] => ['bread'] (Confidence: 0.6666666666666667)
['bread', 'milk'] => ['butter'] (Confidence: 0.6666666666666667)
['butter'] => ['bread', 'milk'] (Confidence: 0.5)
['bread'] => ['butter', 'milk'] (Confidence: 0.5)
['milk'] => ['butter', 'bread'] (Confidence: 0.5)


# Bài 2


**FP-Growth**

In [12]:
class TreeNode:
    def __init__(self, item, frequency, parent):
        self.item = item
        self.frequency = frequency
        self.parent = parent
        self.children = {}
        self.next_link = None

def construct_tree(dataset, min_support):
    header_table = {}
    for transaction in dataset:
        for item in transaction:
            header_table[item] = header_table.get(item, 0) + dataset[transaction]

    for item in list(header_table.keys()):
        if header_table[item] < min_support:
            del(header_table[item])

    frequent_items = set(header_table.keys())

    if len(frequent_items) == 0:
        return None, None

    for item in header_table:
        header_table[item] = [header_table[item], None]

    fp_tree = TreeNode("Null", 1, None)
    for transaction, frequency in dataset.items():
        transaction_sorted = []
        for item in transaction:
            if item in frequent_items:
                transaction_sorted.append(item)
        if len(transaction_sorted) > 0:
            update_tree(fp_tree, transaction_sorted, header_table, frequency)

    return fp_tree, header_table

def update_tree(current_node, transaction_sorted, header_table, frequency):
    if transaction_sorted[0] in current_node.children:
        current_node.children[transaction_sorted[0]].frequency += frequency
    else:
        current_node.children[transaction_sorted[0]] = TreeNode(transaction_sorted[0], frequency, current_node)

        if header_table[transaction_sorted[0]][1] is None:
            header_table[transaction_sorted[0]][1] = current_node.children[transaction_sorted[0]]
        else:
            update_header(header_table[transaction_sorted[0]][1], current_node.children[transaction_sorted[0]])

    if len(transaction_sorted) > 1:
        update_tree(current_node.children[transaction_sorted[0]], transaction_sorted[1:], header_table, frequency)

def update_header(node_to_test, target_node):
    while node_to_test.next_link is not None:
        node_to_test = node_to_test.next_link
    node_to_test.next_link = target_node

def ascend_tree(node, prefix_path):
    if node.parent is not None:
        prefix_path.append(node.item)
        ascend_tree(node.parent, prefix_path)

def find_prefix_path(base_item, header_table):
    tree_node = header_table[base_item][1]
    prefix_paths = {}
    while tree_node is not None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            prefix_paths[frozenset(prefix_path[1:])] = tree_node.frequency
        tree_node = tree_node.next_link
    return prefix_paths

def mine_patterns(header_table, min_support, prefix, frequent_itemsets):
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1][0])]

    for base_item in bigL:
        new_frequent_set = prefix.copy()
        new_frequent_set.add(base_item)
        frequent_itemsets.append(new_frequent_set)
        conditional_tree_paths = find_prefix_path(base_item, header_table)
        conditional_tree, conditional_header = construct_tree(conditional_tree_paths, min_support)
        if conditional_header is not None:
            mine_patterns(conditional_header, min_support, new_frequent_set, frequent_itemsets)

def fpgrowth(dataset, min_support):
    fp_tree, header_table = construct_tree(dataset, min_support)
    frequent_itemsets = []
    mine_patterns(header_table, min_support, set([]), frequent_itemsets)
    return frequent_itemsets

# Tạo tập dữ liệu mẫu
data = [['milk', 'bread', 'butter'],
        ['milk', 'bread'],
        ['milk', 'butter'],
        ['milk', 'bread', 'butter'],
        ['bread', 'butter']]

# Đếm tần suất xuất hiện của mỗi giao dịch
transaction_counts = {}
for transaction in data:
    transaction_counts[frozenset(transaction)] = transaction_counts.get(frozenset(transaction), 0) + 1

# Áp dụng giải thuật FP-Growth
min_support = 2  # Số lần xuất hiện tối thiểu
frequent_itemsets = fpgrowth(transaction_counts, min_support)

print("Độ hỗ trợ của các tập mặt hàng phổ biến:")
for itemset in frequent_itemsets:
    support = support_data[frozenset(itemset)]
    print(f"{itemset}: {support}")


Độ hỗ trợ của các tập mặt hàng phổ biến:
{'butter'}: 0.8
{'bread'}: 0.8
{'butter', 'bread'}: 0.6
{'milk'}: 0.8
{'butter', 'milk'}: 0.6
{'bread', 'milk'}: 0.6
{'butter', 'bread', 'milk'}: 0.4


In [13]:
def generate_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            rules.extend(generate_rules_from_itemset(itemset, min_confidence))
    return rules

def generate_rules_from_itemset(itemset, min_confidence):
    rules = []
    for item in itemset:
        antecedent = frozenset([item])
        consequent = itemset - antecedent
        confidence = support_data[frozenset(itemset)] / support_data[antecedent]
        if confidence >= min_confidence:
            rules.append((antecedent, consequent, confidence))
    return rules

# In ra các luật kết hợp
min_confidence = 0.7  # Ngưỡng độ tin cậy tối thiểu
rules = generate_rules(frequent_itemsets, min_confidence)

print("Các luật kết hợp:")
for antecedent, consequent, confidence in rules:
    print(f"{antecedent} => {consequent} (Confidence: {confidence})")




Các luật kết hợp:
frozenset({'butter'}) => {'bread'} (Confidence: 0.7499999999999999)
frozenset({'bread'}) => {'butter'} (Confidence: 0.7499999999999999)
frozenset({'butter'}) => {'milk'} (Confidence: 0.7499999999999999)
frozenset({'milk'}) => {'butter'} (Confidence: 0.7499999999999999)
frozenset({'bread'}) => {'milk'} (Confidence: 0.7499999999999999)
frozenset({'milk'}) => {'bread'} (Confidence: 0.7499999999999999)


# Bài 3

Lớp Node đại diện cho một nút trong cây. Mỗi nút có một tên (name), trọng số (weight), nút cha (parent) và một từ điển chứa các nút con (children).

Lớp SWNTree đại diện cho cây dựa trên cửa sổ trượt. Nó có một nút gốc (root), một phương thức insert_tree để chèn một giao dịch T vào cây R, một phương thức tw để tính toán trọng số của một giao dịch (được để trống và cần được triển khai), và một phương thức construct để xây dựng cây từ một cơ sở dữ liệu DB.

Phương thức construct sắp xếp các mục trong mỗi giao dịch T theo tần suất trong cửa sổ hiện tại, sau đó chèn T vào cây bằng cách gọi phương thức insert_tree. Cuối cùng, nó trả về nút gốc của cây.



In [14]:
class SWNNode:
    def __init__(self, name):
        self.name = name
        self.weight = 0
        self.pre = 0
        self.pos = 0
        self.parent = None
        self.children = {}

    def insert(self, T, weight):
        if not T:
            return
        item = T[0]
        if item in self.children:
            child = self.children[item]
            child.weight += weight
        else:
            child = SWNNode(item)
            child.weight = weight
            child.parent = self
            self.children[item] = child
        child.insert(T[1:], weight)

class SWNTree:
    def __init__(self):
        self.root = SWNNode('null')

    def construct_swn_tree(self, DB, tw):
        for T in DB:
            T = sorted(T, key=lambda x: x[1], reverse=True)
            self.root.insert([item for item, weight in T], tw)
        self.generate_pre_pos(self.root)

    def generate_pre_pos(self, node, pre=0, pos=0):
        node.pre = pre
        node.pos = pos
        for child in node.children.values():
            pre, pos = self.generate_pre_pos(child, pre + 1, pos)
        node.pos = pos + 1
        return pre, pos + 1
db = [
    [('apple', 2), ('banana', 1), ('orange', 1)],
    [('banana', 3), ('mango', 2)],
    [('apple', 1), ('orange', 2), ('grape', 1)],
    [('banana', 2), ('orange', 1), ('mango', 1)],
    [('apple', 1), ('banana', 1), ('mango', 1), ('grape', 2)]
]


tw = 1
swn_tree = SWNTree()
swn_tree.construct_swn_tree(db, tw)

# Print the SWN tree
def print_tree(node, indent=0):
    print('  ' * indent + f'{node.name} (weight: {node.weight}, pre: {node.pre}, pos: {node.pos})')
    for child in node.children.values():
        print_tree(child, indent + 1)

print_tree(swn_tree.root)

null (weight: 0, pre: 0, pos: 15)
  apple (weight: 1, pre: 1, pos: 3)
    banana (weight: 1, pre: 2, pos: 2)
      orange (weight: 1, pre: 3, pos: 1)
  banana (weight: 2, pre: 4, pos: 7)
    mango (weight: 1, pre: 5, pos: 4)
    orange (weight: 1, pre: 6, pos: 6)
      mango (weight: 1, pre: 7, pos: 5)
  orange (weight: 1, pre: 8, pos: 10)
    apple (weight: 1, pre: 9, pos: 9)
      grape (weight: 1, pre: 10, pos: 8)
  grape (weight: 1, pre: 11, pos: 14)
    apple (weight: 1, pre: 12, pos: 13)
      banana (weight: 1, pre: 13, pos: 12)
        mango (weight: 1, pre: 14, pos: 11)
